In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns

#import pydicom
import time
from functools import partial
import gc
import operator 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
# here you should set which model parameters you want to choose (see definitions.py) and what GPU to use

device=device_by_name("Tesla") # RTX , cpu
torch.cuda.set_device(device)

In [3]:
train_df = pd.read_csv(data_dir+'train.csv')
train_df.shape
train_df=train_df[~train_df.PatientID.isin(bad_images)].reset_index(drop=True)
train_df=train_df.drop_duplicates().reset_index(drop=True)
train_df.shape
train_df.head()

(674252, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


In [12]:
test_df = pd.read_csv(data_dir+'test.csv')
test_df.head()

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


In [5]:
def my_loss(y_pred,y_true,weights):
    window=(y_true>=0).to(torch.float)
    loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*weights.expand_as(y_true)).mean()/(window.mean()+1e-7)
    return loss

In [6]:
class Metric():
    def __init__(self,weights,k=0.03):
        self.weights=weights
        self.k=k
        self.zero()
        
    def zero(self):
        self.loss_sum=0.
        self.loss_count=0.
        self.lossf=0.
        
    def calc(self,y_pred,y_true,prefix=""):
        window=(y_true>=0).to(torch.float)
        loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*self.weights.expand_as(y_true)).mean()/(window.mean()+1e-5)
        self.lossf=self.lossf*(1-self.k)+loss*self.k
        self.loss_sum=self.loss_sum+loss*window.sum()
        self.loss_count=self.loss_count+window.sum()
        return({prefix+'mloss':self.lossf})    
        
    def calc_sums(self,prefix=""):
        return({prefix+'mloss_tot':self.loss_sum/self.loss_count})    



In [7]:
model_names=[]
types_train=[]
types_test=[]
versions=[]
num_splits =[]
seeds=[]
for key in parameters.keys():
    model_names.append(parameters[key]['model_name'])
    types_train.append(parameters[key]['train_features'])
    types_test.append(parameters[key]['test_features'])
    versions.append(parameters[key]['version'])
    num_splits.append(parameters[key]['n_splits'])
    seeds.append(parameters[key]['SEED'])   

In [ ]:
multi=3
for model_name,type_,version_,n,SEED in zip(model_names,types_train,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        pickle_file=open(outputs_dir+'PID_splits_{}.pkl'.format(n_splits),'rb')
        split_sid,splits=pickle.load(pickle_file)
        pickle_file.close()
        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//4,4,-1)
        split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =3)

        win_dataset=FullHeadDataset(train_df,
                                      split_validate,
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'OOF_pred',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


In [ ]:
multi=3
for model_name,type_,version_,n,SEED in zip(model_names,types_test,versions,num_splits,seeds):
    for num_split in tqdm_notebook(range(n)):
        pred_list=[]
        print(model_name,version_,type_,num_split) 
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version_,type_,num_split),'rb')
        features=pickle.load(pickle_file)
        pickle_file.close()
        features=features.reshape(features.shape[0]//8,8,-1)
        print(features.shape)
        model=ResModelPool(features.shape[-1])
        version=version_+'_fullhead_resmodel_pool2_{}'.format(multi)

        model.load_state_dict(torch.load(models_dir+models_format.format(model_name,version,num_split),map_location=torch.device(device)))

        valid_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                      multi =4)

        win_dataset=FullHeadDataset(test_df,
                                      test_df.SeriesI.unique(),
                                      features,
                                      'SeriesI',
                                      'ImagePositionZ',
                                       target_columns=hemorrhage_types)
        win_list=[]
        dl = D.DataLoader(win_dataset,batch_size=128,num_workers=16)
        for _,win in tqdm_notebook(dl):
            win_list.append(win.reshape(win.shape[0]*win.shape[1],-1))    
        wins = torch.cat(win_list,0).sum(1)>=0
        wins.sum()
        for i in tqdm_notebook(range(32),leave=False):
            pr = model_run(model,valid_dataset,do_apex=False,batch_size=128)
            pred_list.append(pr.reshape(pr.shape[0]*pr.shape[1],-1)[wins])
        pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'test_pred_ensemble',num_split),'wb')
        pickle.dump(pred_list,pickle_file,protocol=4)
        pickle_file.close()


In [ ]:
OOF_ids={}
SEED=8153
n_splits=3
pickle_file=open(outputs_dir+'PID_splits_{}.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()
for i in range( n_splits):
    images_id_list=[]
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].SeriesI.unique()
    image_arr=train_df.PatientID.values
    ref_arr=train_df.SeriesI.values
    order_arr=train_df.ImagePositionZ.values
    for s in tqdm_notebook(split_validate):
        head_idx = np.where(ref_arr==s)[0]
        sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
        images_id_list.append(image_arr[sorted_head_idx])
    image_ids=np.concatenate(images_id_list)
    print(image_ids.shape,train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].shape[0])
    OOF_ids[i]=image_ids


pickle_file=open(outputs_dir+'OOF_validation_image_ids_{}.pkl'.format(n_splits),'wb')
pickle.dump(OOF_ids,pickle_file,protocol=4)
pickle_file.close()


In [ ]:
OOF_ids={}
SEED=432
n_splits=5
pickle_file=open(outputs_dir+'PID_splits_{}.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()
for i in range( n_splits):
    images_id_list=[]
    split_validate =  train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].SeriesI.unique()
    image_arr=train_df.PatientID.values
    ref_arr=train_df.SeriesI.values
    order_arr=train_df.ImagePositionZ.values
    for s in tqdm_notebook(split_validate):
        head_idx = np.where(ref_arr==s)[0]
        sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
        images_id_list.append(image_arr[sorted_head_idx])
    image_ids=np.concatenate(images_id_list)
    print(image_ids.shape,train_df[train_df.PID.isin(set(split_sid[splits[i][1]]))].shape[0])
    OOF_ids[i]=image_ids


pickle_file=open(outputs_dir+'OOF_validation_image_ids_{}.pkl'.format(n_splits),'wb')
pickle.dump(OOF_ids,pickle_file,protocol=4)
pickle_file.close()


In [14]:
images_id_list=[]
dummeys=[]
image_arr=test_df.PatientID.values
ref_arr=test_df.SeriesI.values
order_arr=test_df.ImagePositionZ.values
for s in tqdm_notebook(test_df.SeriesI.unique()):
    dumm=np.zeros(60)
    head_idx = np.where(ref_arr==s)[0]
    sorted_head_idx=head_idx[np.argsort(order_arr[head_idx])]
    images_id_list.append(image_arr[sorted_head_idx])
    dumm[0:head_idx.shape[0]]=1
    dummeys.append(dumm)
image_ids=np.concatenate(images_id_list)
select=np.concatenate(dummeys)==1

pickle_file=open(outputs_dir+'ensemble_test_image_ids.pkl','wb')
pickle.dump(image_ids,pickle_file,protocol=4)
pickle_file.close()
